# Using MLRUN with Nuclio
WORK IN PROGRESS: add the `handler` wrapper automatically 

In [1]:
# nuclio: ignore
import nuclio

In [2]:
from mlrun import get_or_create_ctx, RunTemplate, NewRun, runtimes
import time

def my_func(context, p1=1, p2='txt'):
    context.logger.info(
        f'Run: {context.name} uid={context.uid}:{context.iteration} Params: p1={p1}, p2={p2}')

    time.sleep(1)

    # log scalar values (KFP metrics)
    context.log_result('accuracy', p1 * 2)
    context.log_result('latency', p1 * 3)

    # log various types of artifacts (and set UI viewers)
    context.log_artifact('test.txt', body=b'abc is 123')
    context.log_artifact('test.html', body=b'<b> Some HTML <b>', viewer='web-app')

    context.logger.info('run complete!')
    return 'func-resp'

def handler(context, event):
    ctx = get_or_create_ctx('myfunc', event=event)
    args = runtimes.local.get_func_arg(my_func, RunTemplate.from_dict(ctx.to_dict()), ctx)

    try:
        val = my_func(*args)
        if val:
            ctx.log_result('return', val)
    except Exception as e:
        err = str(e)
        ctx.set_state(error=err)
    return ctx.to_json()

In [3]:
# nuclio: end-code
from mlrun import new_function
runner = new_function()
list_run = runner.run(handler=handler, params={'p1': 7})

[mlrun] 2019-09-07 22:38:26,588 starting run None uid=884b6339c87b400692164802ca3ecb61
[mlrun] 2019-09-07 22:38:27,467 Run: myfunc uid=ca8dab348ae844888e54670f0b4f5bab:0 Params: p1=1, p2=txt
[mlrun] 2019-09-07 22:38:28,472 run complete!



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...3ecb61,0,Sep 07 22:38:27,completed,,kind=handlerowner=iguaziohost=jupyter-rngbsdr6ab-f02en-5dd7cd96db-xfzxr,,p1=7,"return={""metadata"": {""name"": ""myfunc"", ""uid"": ""ca8dab348ae844888e54670f0b4f5bab"", ""iteration"": 0, ""project"": """", ""labels"": {""host"": ""jupyter-rngbsdr6ab-f02en-5dd7cd96db-xfzxr""}, ""annotations"": {}}, ""spec"": {""runtime"": {}, ""log_level"": ""info"", ""parameters"": {}, ""inputs"": {}, ""data_stores"": [], ""output_artifacts"": [], ""output_path"": """"}, ""status"": {""state"": ""running"", ""outputs"": {""accuracy"": 2, ""latency"": 3, ""return"": ""func-resp""}, ""start_time"": ""2019-09-07 22:38:27.466914"", ""last_update"": ""2019-09-07 22:38:27.466916"", ""host"": ""jupyter-rngbsdr6ab-f02en-5dd7cd96db-xfzxr"", ""output_artifacts"": [{""key"": ""test.txt"", ""kind"": """", ""tree"": ""ca8dab348ae844888e54670f0b4f5bab"", ""target_path"": ""test.txt"", ""hash"": ""8170b9a53bbb1f4d52733bc6824955e3a362d4a1""}, {""key"": ""test.html"", ""kind"": """", ""tree"": ""ca8dab348ae844888e54670f0b4f5bab"", ""target_path"": ""test.html"", ""hash"": ""11f50bed0aa902cbdcf3249c610af88bc26f1619"", ""viewer"": ""web-app""}]}}",


[mlrun] 2019-09-07 22:38:28,496 run executed, status=created


In [4]:
spec = nuclio.ConfigSpec(env={'MYENV_VAR': 'something'}, 
                         cmd=['pip install git+https://github.com/mlrun/mlrun.git@development'],
                         config={'spec.build.baseImage': 'python:3.6-jessie', 'spec.build.noCache': True},
                         mount=nuclio.Volume('User','~/'))
spec.add_trigger('web', nuclio.triggers.HttpTrigger(workers=8))

addr = nuclio.deploy_file('',name='myfunc', project='mlrun', verbose=False, spec=spec)

[nuclio.deploy] 2019-09-07 22:38:51,898 (info) Building processor image
[nuclio.deploy] 2019-09-07 22:39:31,178 (info) Pushing image
[nuclio.deploy] 2019-09-07 22:39:42,259 (info) Build complete
[nuclio.deploy] 2019-09-07 22:39:46,287 (info) Function deploy complete
[nuclio.deploy] 2019-09-07 22:39:46,292 done updating myfunc, function address: 18.188.201.226:30493


In [5]:
resp = new_function(command=f'http://{addr}').run(params={'p1':9})

[mlrun] 2019-09-07 22:39:46,297 starting run None uid=4ffbb47c6606414981a027f49f6e0d36



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...6e0d36,0,Sep 07 22:39:46,completed,myfunc,kind=remoteowner=iguaziohost=myfunc-6748776b6f-k6sgp,,p1=9,accuracy=18latency=27return=func-resp,test.txttest.html


[mlrun] 2019-09-07 22:39:47,328 run executed, status=created
